In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as ds

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.optimizers import Adam

In [3]:
(x_train,y_train),(x_test,y_test)=ds.mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

In [4]:
cnn=Sequential()
cnn.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
cnn.add(Conv2D(32,(3,3),activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(units=512,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(units=10,activation='softmax'))

In [5]:
cnn.compile(loss='categorical_crossentropy',optimizer=Adam
(learning_rate=0.001),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test),verbose=2)

Epoch 1/100
469/469 - 13s - loss: 0.2362 - accuracy: 0.9250 - val_loss: 0.0448 - val_accuracy: 0.9853 - 13s/epoch - 28ms/step
Epoch 2/100
469/469 - 13s - loss: 0.0651 - accuracy: 0.9795 - val_loss: 0.0335 - val_accuracy: 0.9895 - 13s/epoch - 28ms/step
Epoch 3/100
469/469 - 13s - loss: 0.0491 - accuracy: 0.9848 - val_loss: 0.0261 - val_accuracy: 0.9918 - 13s/epoch - 27ms/step
Epoch 4/100
469/469 - 12s - loss: 0.0396 - accuracy: 0.9879 - val_loss: 0.0212 - val_accuracy: 0.9936 - 12s/epoch - 26ms/step
Epoch 5/100
469/469 - 12s - loss: 0.0343 - accuracy: 0.9898 - val_loss: 0.0199 - val_accuracy: 0.9929 - 12s/epoch - 26ms/step
Epoch 6/100
469/469 - 12s - loss: 0.0298 - accuracy: 0.9904 - val_loss: 0.0199 - val_accuracy: 0.9943 - 12s/epoch - 26ms/step
Epoch 7/100
469/469 - 12s - loss: 0.0271 - accuracy: 0.9915 - val_loss: 0.0172 - val_accuracy: 0.9940 - 12s/epoch - 26ms/step
Epoch 8/100
469/469 - 12s - loss: 0.0251 - accuracy: 0.9920 - val_loss: 0.0168 - val_accuracy: 0.9944 - 12s/epoch - 25

In [6]:
cnn.save('cnn_v2.h5')

In [7]:
res=cnn.evaluate(x_test,y_test,verbose=0)
print('정확률=', res[1]*100)

정확률= 99.55999851226807


In [10]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import winsound

# 모델 로드
model = tf.keras.models.load_model('cnn_v2.h5')

user_input_numbers = []
predicted_numbers = []  # 전역 변수로 이동

def reset():
    global img
    img = np.ones((200, 520, 3), dtype=np.uint8) * 255
    for i in range(5):
        cv.rectangle(img, (10 + i * 100, 50), (10 + (i + 1) * 100, 150), (0, 0, 255))
    cv.putText(img, 'e:erase s:show r:recognition q:quit', (10, 40), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)

def grab_numerals():
    numerals = []
    for i in range(5):
        roi = img[50:150, 10 + i * 100:110 + i * 100, 0]
        roi = 255 - cv.resize(roi, (28, 28), interpolation=cv.INTER_CUBIC)
        numerals.append(roi)
    numerals = np.array(numerals)
    return numerals

def show():
    numerals = grab_numerals()
    plt.figure(figsize=(25, 5))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(numerals[i], cmap='gray')
        plt.xticks([]); plt.yticks([])
    plt.show()

def recognition():
    global predicted_numbers  # 전역 변수 사용
    numerals = grab_numerals()
    numerals = numerals.reshape(5, 28, 28,1)
    numerals = numerals.astype(np.float32) / 255.0
    res = model.predict(numerals)
    class_id = np.argmax(res, axis=1)
    predicted_numbers = class_id.tolist()  # 예측 결과 저장
    for i in range(5):
        cv.putText(img, str(class_id[i]), (50 + i * 100, 180), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
    winsound.Beep(1000, 500)

BrushSiz = 4
LColor = (0, 0, 0)

def writing(event, x, y, flags, param):
    if event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        cv.circle(img, (x, y), BrushSiz, LColor, -1)

reset()
cv.namedWindow('Writing')
cv.setMouseCallback('Writing', writing)

while True:
    cv.imshow('Writing', img)
    key = cv.waitKey(1)
    if key == ord('e'):
        reset()
    elif key == ord('s'):
        show()
    elif key == ord('r'):
        recognition()
    elif key == ord('q'):
        break

cv.destroyAllWindows()

# 사용자에게 입력 숫자 질문
for i in range(5):
    num = int(input(f"Enter the number at position {i+1}: "))
    user_input_numbers.append(num)

# 정답 개수 계산
correct_count = sum([1 for i in range(5) if user_input_numbers[i] == predicted_numbers[i]])
print(f"You got {correct_count} out of 5 correct.")
print("정답률: ",correct_count/5*100,"%" )

1/1 [==============================] - 0s 54ms/step
You got 4 out of 5 correct.
정답률:  80.0 %
